In [ ]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm
- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download
English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/
# References
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
from data_handler import *
import time
import scipy

Using TensorFlow backend.


In [2]:
num_frames = 20
batch_size =  10000
image_size =  64
num_digits = 2
step_length=  0.1
    
#num_frames, batch_size, image_size, num_digits, step_length
data_param = [num_frames, batch_size, image_size, num_digits, step_length]
train_data = ChooseDataHandler(data_param)

VALUE IS  <HDF5 dataset "train": shape (50000, 784), type "<f4">
SHAPE OF data is  (50000, 28, 28)


In [3]:
v_cpu, _ = train_data.GetBatch()
print(v_cpu.shape)

Get Batch inside the bounding 20
(10000, 81920)


In [4]:
batch_size = 80  # Batch size for training.
epochs     = 10    # Number of epochs to train for.
latent_dim = 2048  # Latent dimensionality of the encoding space.
num_samples = 100  # Number of samples to train on.


max_encoder_seq_length = 10
max_decoder_seq_length = 10
num_encoder_tokens = 4096
num_decoder_tokens = 4096
input_texts = 1

In [12]:
v_cpua = v_cpu.reshape((10000, 20, 64, 64))


In [5]:
v_cpua = v_cpu.reshape((10000, 20, 64*64))

In [14]:
counter  = 0
for t in v_cpua[0]:
    scipy.misc.imsave(str(counter)+'.jpg', t)
    counter = counter+1

/home/pranjal/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
encoder_input_data = v_cpua[0, 0:10, :]
encoder_input_data = np.expand_dims(encoder_input_data, axis=0)
print(encoder_input_data.shape)

(1, 10, 4096)


In [7]:
decoder_input_data = encoder_input_data
decoder_target_data = decoder_input_data

In [ ]:

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [8]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [9]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=1)#,
          #validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/1
1/1 [==============================] - 28s - loss: 1742.5747


In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [11]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    decoded_sentence = []
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    for idk in range(0, 10):
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        decoded_sentence.append(output_tokens)
        target_seq = output_tokens
        # Update states
        states_value = [h, c]
    return decoded_sentence

# for seq_index in range(100):
#     # Take one sequence (part of the training test)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
    
#     start = time.time()
#     decoded_sentence = decode_sequence(input_seq)
#     timespent  = time.time() - start
#     print('TIME SPENT IS ', timespent)

In [22]:
input_seq = encoder_input_data#encoder_input_data[seq_index: seq_index + 1]
start = time.time()
decoded_sentence = decode_sequence(input_seq)
print(decoded_sentence[0], len(decoded_sentence))
endtime = time.time()
timespent  = endtime - start
print(start, endtime)
print (time.localtime( start ))
print (time.localtime( endtime ))
print(timespent)

[[[ 0.00014355  0.00013044  0.00022725 ...,  0.00022894  0.00012799
    0.00026245]]] 10
1512245662.6711671 1512245662.7445028
time.struct_time(tm_year=2017, tm_mon=12, tm_mday=2, tm_hour=15, tm_min=14, tm_sec=22, tm_wday=5, tm_yday=336, tm_isdst=0)
time.struct_time(tm_year=2017, tm_mon=12, tm_mday=2, tm_hour=15, tm_min=14, tm_sec=22, tm_wday=5, tm_yday=336, tm_isdst=0)
0.07333564758300781
